### Inspired from

- https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
- https://www.kaggle.com/code/ibtesama/gan-in-pytorch-with-fid

### Training logs
1. https://wandb.ai/ahmadjaved97/dcgan-t1?nw=nwuserahmadjaved97
2. https://wandb.ai/ahmadjaved97/dcgan-t2?nw=nwuserahmadjaved97

### Import required libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import StepLR
from torchvision.utils import make_grid, save_image
from torchmetrics.image.fid import FrechetInceptionDistance
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import random
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
import seaborn as sns
import wandb
import imageio
import datetime

### Set Device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device Used: {device}")

Device Used: cuda


In [3]:
saved_model_folder = "./saved_models"
os.makedirs(saved_model_folder, exist_ok=True)

### Data Loader Class
This is same as the one used in **Assignment-5**

In [4]:
class AFHQDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        root_dir: path of the parent directory that contains images.
        transforms: augmentations applied to the images (can be none or more).
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_mapping = {}
        
        extensions = (".jpg", ".jpeg", ".png")
        # go through all sub-directories
        for label, category in enumerate(sorted(os.listdir(root_dir))):
            full_path = os.path.join(root_dir, category)
            if os.path.exists(full_path):
                self.class_mapping[label] = category
                for img_name in os.listdir(full_path):
                    if img_name.endswith(extensions):
                        self.image_paths.append(os.path.join(full_path, img_name))
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [5]:
# Dataset Hyperparameters
img_size = 64
batch_size = 64

# dataset paths
train_dir = '/home/user/javeda1/stargan-v2/data/afhq/train'
val_dir = '/home/user/javeda1/stargan-v2/data/afhq/val'

In [6]:
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [7]:
# Load the train and val dataset
train_dataset = AFHQDataset(root_dir=train_dir, transform=transform)
val_dataset = AFHQDataset(root_dir=val_dir, transform=transform)

# DataLoaders for train and val sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

train_size = len(train_loader.dataset)
val_size = len(val_loader.dataset)

print(f"Train dataset size: {train_size}")
print(f"Validation dataset size: {val_size}")

Train dataset size: 14630
Validation dataset size: 1500


In [8]:
# Same as the one used in assignment-5
def compute_fid_score_gan(generator, data_loader, device, latent_dim=256, max_samples=1000):
    """
    Computes the Fréchet Inception Distance (FID) between real and generated images.
    """
    fid = FrechetInceptionDistance(feature=2048).to(device)
    generator.eval()
    
    total_samples = 0
    with torch.no_grad():
        for real_images, _ in data_loader:
            batch_size = real_images.size(0)
            real_images = real_images.to(device)
            
            # Generate fake images
            noise = torch.randn(batch_size, latent_dim, device=device)
            fake_images = generator(noise)
            
            # Convert images to uint8 and scale to [0, 255]
            real_images = (real_images * 0.5 + 0.5).clamp(0, 1)
            fake_images = (fake_images * 0.5 + 0.5).clamp(0, 1)
            real_images = (real_images * 255).byte()
            fake_images = (fake_images * 255).byte()
            
            fid.update(real_images, real=True)
            fid.update(fake_images, real=False)
            
            total_samples += batch_size
            if total_samples >= max_samples:
                break
    
    return fid.compute().item()

In [9]:
def train_dcgan(generator, discriminator, dataloader, latent_dim, epochs, device,
                train_disc=1, lr_g=0.0002, lr_d=0.0008, beta1=0.5, step_size=10, gamma=0.5,
                label_smoothing=False, project="DCGAN-AFHQ2", name="run1",
               output_folder="generated_images"):
    
    #optimizers
    optimizer_g = optim.Adam(generator.parameters(), lr=lr_g, betas=(beta1, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=lr_d, betas=(beta1, 0.999))

    # Learning rate schedulers
    scheduler_g = optim.lr_scheduler.StepLR(optimizer_g, step_size=step_size, gamma=gamma)
    scheduler_d = optim.lr_scheduler.StepLR(optimizer_d, step_size=step_size, gamma=gamma)

    # Loss function
    criterion = nn.BCELoss()

    # Fixed noise for evaluation
    fixed_noise = torch.randn(64, latent_dim, device=device)

    # Labels for real and fake data
    if label_smoothing:
        real_label = 0.9  # Label smoothing for real images
    else:
        real_label = 1.0

    fake_label = 0.0

    # Initialize WandB
    run_timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    unique_name = f"{name}_{run_timestamp}"

    wandb.init(project=project, name=name, id=unique_name, config={
        "latent_dim": latent_dim,
        "epochs": epochs,
        "lr_g": lr_g,
        "lr_d": lr_d,
        "beta1": beta1,
        "step_size": step_size,
        "gamma": gamma
    })

    # Output directory for generated images
    output_dir = output_folder
    os.makedirs(output_dir, exist_ok=True)

    # Training loop
    for epoch in range(epochs):
        g_loss_total = 0.0
        d_loss_total = 0.0

        for i, (real_images, _) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")):
            batch_size = real_images.size(0)

            real_images = real_images.to(device)

            # Train Discriminator
            if i % train_disc == 0:  # Train the discriminator less frequently
                discriminator.zero_grad()

                # Real images
                labels_real = torch.full((batch_size,), real_label, device=device)
                output_real = discriminator(real_images).view(-1)
                loss_real = criterion(output_real, labels_real)

                # Fake images
                noise = torch.randn(batch_size, latent_dim, device=device)
                fake_images = generator(noise)
                labels_fake = torch.full((batch_size,), fake_label, device=device)
                output_fake = discriminator(fake_images.detach()).view(-1)
                loss_fake = criterion(output_fake, labels_fake)

                # Combine losses and backpropagate
                d_loss = loss_real + loss_fake
                d_loss.backward()
                optimizer_d.step()

                d_loss_total += d_loss.item()

            # Train Generator
            generator.zero_grad()
            noise = torch.randn(batch_size, latent_dim, device=device)
            fake_images = generator(noise)
            output = discriminator(fake_images).view(-1)
            g_loss = criterion(output, labels_real)
            g_loss.backward()
            optimizer_g.step()

            g_loss_total += g_loss.item()

        # Step LR
        scheduler_g.step()
        scheduler_d.step()

        # Log learning rates
        # wandb.log({"Learning Rate Generator": scheduler_g.get_last_lr()[0],
        #            "Learning Rate Discriminator": scheduler_d.get_last_lr()[0]})

        # Log losses and generate sample images
        avg_g_loss = g_loss_total / len(dataloader)
        avg_d_loss = d_loss_total / len(dataloader)

        # Compute FID
        fid_score = compute_fid_score_gan(generator, dataloader, device, latent_dim)

        # Log metrics
        # wandb.log({"Generator Loss": avg_g_loss, "Discriminator Loss": avg_d_loss, "FID Score": fid_score})

        # Genreate fake image as grids
        with torch.no_grad():
            sample_images = generator(fixed_noise).detach().cpu()
            sample_grid = make_grid(sample_images, nrow=8, normalize=True, scale_each=True)

            real_images_grid = make_grid(real_images[:64], nrow=8, normalize=True, scale_each=True)

            # Save generated grid to disk
            save_image(sample_grid, os.path.join(output_dir, f"epoch_{epoch+1:03d}_generated.png"))

        wandb.log({
            "Generated Images Grid": wandb.Image(sample_grid, caption="Generated Images"),
            "Real Images Grid": wandb.Image(real_images_grid, caption="Real Images"),
            "Learning Rate Generator": scheduler_g.get_last_lr()[0],
            "Learning Rate Discriminator": scheduler_d.get_last_lr()[0],
            "Generator Loss": avg_g_loss, "Discriminator Loss": avg_d_loss, "FID Score": fid_score
        })

    # Finish the WandB run
    wandb.finish()
    return

### Define Model

In [10]:
# Taken from: https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
# Define weights initialization function
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [11]:
class Generator(nn.Module):
    def __init__(self, latent_dim=256, num_channels=3):
        super(Generator, self).__init__()
        
        
        self.fc = nn.Linear(latent_dim, 512 * 4 * 4)
        self.bn1 = nn.BatchNorm2d(512)
        self.relu1 = nn.ReLU(True)
        
        # Second block
        self.upsample2 = nn.Upsample(scale_factor=2)
        self.conv2 = nn.Conv2d(512, 256, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(256)
        self.relu2 = nn.ReLU(True)
        
        # Third block
        self.upsample3 = nn.Upsample(scale_factor=2)
        self.conv3 = nn.Conv2d(256, 128, 3, 1, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU(True)
        
        # Fourth block
        self.upsample4 = nn.Upsample(scale_factor=2)
        self.conv4 = nn.Conv2d(128, 64, 3, 1, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(64)
        self.relu4 = nn.ReLU(True)
        
        # Fifth block
        self.upsample5 = nn.Upsample(scale_factor=2)
        self.conv5 = nn.Conv2d(64, num_channels, 3, 1, 1, bias=False)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.fc(x.view(-1, x.size(1)))
        x = x.view(-1, 512, 4, 4)
        x = self.bn1(x)
        x = self.relu1(x)
        
        # Second block
        x = self.upsample2(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        
        # Third block
        x = self.upsample3(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        
        # Fourth block
        x = self.upsample4(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        
        # Fifth block
        x = self.upsample5(x)
        x = self.conv5(x)
        x = self.tanh(x)
        
        return x

class Discriminator(nn.Module):
    def __init__(self, num_channels=3):
        super(Discriminator, self).__init__()
        
        # First convolution block
        self.conv1 = nn.Conv2d(num_channels, 64, 4, 2, 1, bias=False)
        self.lrelu1 = nn.LeakyReLU(0.2, inplace=True)
        
        # Second convolution block
        self.conv2 = nn.Conv2d(64, 128, 4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(128)
        self.lrelu2 = nn.LeakyReLU(0.2, inplace=True)
        
        # Third convolution block
        self.conv3 = nn.Conv2d(128, 256, 4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(256)
        self.lrelu3 = nn.LeakyReLU(0.2, inplace=True)
        
        # Fourth convolution block
        self.conv4 = nn.Conv2d(256, 512, 4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(512)
        self.lrelu4 = nn.LeakyReLU(0.2, inplace=True)
        
        # Final convolution
        self.conv5 = nn.Conv2d(512, 1, 4, 1, 0, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.lrelu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.lrelu2(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.lrelu3(x)
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.lrelu4(x)
        
        x = self.conv5(x)
        x = self.sigmoid(x)
        
        return x.view(-1)

### Model Training

In [ ]:
# Train the model
latent_dim = 100
epochs = 25
lr_g=0.0002
lr_d=0.0004
train_disc=1
label_smoothing=True
generator = Generator(latent_dim=latent_dim).to(device)
generator.apply(weights_init)
discriminator = Discriminator().to(device)
discriminator.apply(weights_init)
name=f"run_ld_{latent_dim}_ep_{epochs}_td_{train_disc}"
train_dcgan(generator, discriminator, train_loader, latent_dim, epochs, device, lr_g=lr_g, lr_d=lr_d, project='dcgan-t1', name=name, label_smoothing=label_smoothing)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ahmadjaved97. Use `wandb login --relogin` to force relogin


Epoch 9/25: 100%|█████████████████████████████████████████| 229/229 [00:18<00:00, 12.11it/s]


In [14]:
# Train the model
latent_dim = 128
epochs = 25
lr_g=0.0002
lr_d=0.0004
train_disc=1
generator = Generator(latent_dim=latent_dim).to(device)
generator.apply(weights_init)
discriminator = Discriminator().to(device)
discriminator.apply(weights_init)
name=f"run_ld_{latent_dim}_ep_{epochs}_td_{train_disc}"
label_smoothing=True
train_dcgan(generator, discriminator, train_loader, latent_dim, epochs, device, lr_g=lr_g, lr_d=lr_d, project='dcgan-t1', name=name, label_smoothing=label_smoothing)

Epoch 25/25: 100%|████████████████████████████████████████| 229/229 [00:09<00:00, 23.38it/s]


Discriminator Loss,█▆▃▄▄▄▄▃▃▃▂▂▂▂▂▃▂▂▂▂▁▁▁▁▁
FID Score,██▇▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▂▁
Generator Loss,█▂▅▂▃▂▂▁▂▂▂▂▂▂▃▂▂▃▃▃▃▃▃▄▄
Learning Rate Discriminator,█████████▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁
Learning Rate Generator,█████████▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁
Discriminator Loss,0.39326
FID Score,120.48156
Generator Loss,4.13669
Learning Rate Discriminator,0.0001
Learning Rate Generator,5e-05


In [16]:
# Train the model
latent_dim = 200
epochs = 25
lr_g=0.0002
lr_d=0.0004
train_disc=1
generator = Generator(latent_dim=latent_dim).to(device)
generator.apply(weights_init)
discriminator = Discriminator().to(device)
discriminator.apply(weights_init)
name=f"run_ld_{latent_dim}_ep_{epochs}_td_{train_disc}"
output_folder=name
label_smoothing=True
train_dcgan(generator, discriminator, train_loader, latent_dim, epochs, device, lr_g=lr_g, lr_d=lr_d, project='dcgan-t1', name=name,
           output_folder=name, label_smoothing=label_smoothing)

Epoch 25/25: 100%|████████████████████████████████████████| 229/229 [00:10<00:00, 22.39it/s]


Discriminator Loss,█▅▅▄▅▄▄▃▄▃▁▁▂▁▂▁▂▁▃▁▁▁▁▁▁
FID Score,█▇▆▇▆▆▆▅▄▅▄▃▃▃▃▃▃▂▂▂▁▁▁▁▁
Generator Loss,█▂▃▂▁▁▂▂▁▃▁▂▄▃▄▄▄▄▄▄▄▄▅▅▆
Learning Rate Discriminator,█████████▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁
Learning Rate Generator,█████████▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁
Discriminator Loss,0.05287
FID Score,116.18854
Generator Loss,6.10066
Learning Rate Discriminator,0.0001
Learning Rate Generator,5e-05


In [17]:
# Train the model
latent_dim = 256
epochs = 25
lr_g=0.0002
lr_d=0.0004
train_disc=1
generator = Generator(latent_dim=latent_dim).to(device)
generator.apply(weights_init)
discriminator = Discriminator().to(device)
discriminator.apply(weights_init)
name=f"run_ld_{latent_dim}_ep_{epochs}_td_{train_disc}"
output_folder=name
label_smoothing=True
train_dcgan(generator, discriminator, train_loader, latent_dim, epochs, device, lr_g=lr_g, lr_d=lr_d, project='dcgan-t1', name=name,
           output_folder=name)

Epoch 25/25: 100%|████████████████████████████████████████| 229/229 [00:18<00:00, 12.18it/s]


Discriminator Loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
FID Score,▁█▇▆▆▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Generator Loss,▁▅▅▅▅▆███████████████████
Learning Rate Discriminator,█████████▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁
Learning Rate Generator,█████████▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁
Discriminator Loss,0.0
FID Score,331.23126
Generator Loss,21.93337
Learning Rate Discriminator,0.0001
Learning Rate Generator,5e-05


#### Here the discriminator is trained less frequently than the generator

In [12]:
# Train the model
latent_dim = 200
epochs = 25
lr_g=0.0002
lr_d=0.0004
train_disc=2
generator = Generator(latent_dim=latent_dim).to(device)
generator.apply(weights_init)
discriminator = Discriminator().to(device)
discriminator.apply(weights_init)
name=f"run_ld_{latent_dim}_ep_{epochs}_td_{train_disc}"
output_folder=name
label_smoothing=True
train_dcgan(generator, discriminator, train_loader, latent_dim, epochs, device, lr_g=lr_g, lr_d=lr_d, project='dcgan-t2', name=name,
           output_folder=name, label_smoothing=label_smoothing)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ahmadjaved97. Use `wandb login --relogin` to force relogin


Epoch 25/25: 100%|████████████████████████████████████████| 229/229 [00:10<00:00, 22.57it/s]


Discriminator Loss,█▄▅▄▄▄▃▃▄▃▂▂▃▂▂▂▂▂▂▂▁▁▁▁▁
FID Score,▇█▇▇▇▆▆▆▅▅▅▄▄▅▄▃▂▃▂▂▁▁▁▂▁
Generator Loss,█▄▃▃▃▂▃▃▁▂▁▂▂▃▃▄▄▄▄▄▃▄▅▅▅
Learning Rate Discriminator,█████████▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁
Learning Rate Generator,█████████▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁
Discriminator Loss,0.39584
FID Score,131.54034
Generator Loss,4.15331
Learning Rate Discriminator,0.0001
Learning Rate Generator,5e-05


In [13]:
# Train the model
latent_dim = 200
epochs = 35
lr_g=0.0002
lr_d=0.0004
train_disc=2
generator = Generator(latent_dim=latent_dim).to(device)
generator.apply(weights_init)
discriminator = Discriminator().to(device)
discriminator.apply(weights_init)
name=f"run_ld_{latent_dim}_ep_{epochs}_td_{train_disc}"
output_folder=name
label_smoothing=True
train_dcgan(generator, discriminator, train_loader, latent_dim, epochs, device, lr_g=lr_g, lr_d=lr_d, project='dcgan-t2', name=name,
           output_folder=name, label_smoothing=label_smoothing)

Epoch 35/35: 100%|████████████████████████████████████████| 229/229 [00:09<00:00, 23.24it/s]


Discriminator Loss,█▅▄▃▄▃▄▃▃▃▂▂▂▂▂▂▂▂▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
FID Score,███▇▇▆▆▆▆▅▄▄▄▅▃▃▃▃▃▃▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
Generator Loss,▅▃▄▄▂▂▁▂▂▂▂▃▃▃▄▄▄▅▃▄▄▄▄▅▅▆▆▆▆▆▇▇███
Learning Rate Discriminator,█████████▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
Learning Rate Generator,█████████▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
Discriminator Loss,0.36023
FID Score,114.40671
Generator Loss,4.60616
Learning Rate Discriminator,5e-05
Learning Rate Generator,3e-05


In [14]:
# Train the model
latent_dim = 200
epochs = 40
lr_g=0.0002
lr_d=0.0004
train_disc=3
generator = Generator(latent_dim=latent_dim).to(device)
generator.apply(weights_init)
discriminator = Discriminator().to(device)
discriminator.apply(weights_init)
name=f"run_ld_{latent_dim}_ep_{epochs}_td_{train_disc}"
output_folder=name
label_smoothing=True
train_dcgan(generator, discriminator, train_loader, latent_dim, epochs, device, lr_g=lr_g, lr_d=lr_d, project='dcgan-t2', name=name,
           output_folder=name, label_smoothing=label_smoothing)

Epoch 40/40: 100%|████████████████████████████████████████| 229/229 [00:09<00:00, 24.78it/s]


Discriminator Loss,█▄▄▄▄▄▃▃▄▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
FID Score,█▇█▇▆▆▇▇▆▅▄▄▄▄▄▃▅▃▃▂▂▂▂▂▂▂▁▂▂▁▁▂▁▁▁▁▁▁▁▁
Generator Loss,▅▄▃▃▃▃▂▂▁▂▁▂▂▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇██████
Learning Rate Discriminator,█████████▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Learning Rate Generator,█████████▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Discriminator Loss,0.36391
FID Score,126.20745
Generator Loss,4.75845
Learning Rate Discriminator,3e-05
Learning Rate Generator,1e-05


In [15]:
# Train the model
latent_dim = 200
epochs = 40
lr_g=0.0002
lr_d=0.0004
train_disc=3
generator = Generator(latent_dim=latent_dim).to(device)
generator.apply(weights_init)
discriminator = Discriminator().to(device)
discriminator.apply(weights_init)
name=f"run_ld_{latent_dim}_ep_{epochs}_td_{train_disc}_no_ls"
output_folder=name
label_smoothing=False
train_dcgan(generator, discriminator, train_loader, latent_dim, epochs, device, lr_g=lr_g, lr_d=lr_d, project='dcgan-t2', name=name,
           output_folder=name, label_smoothing=label_smoothing)

Epoch 40/40: 100%|████████████████████████████████████████| 229/229 [00:11<00:00, 20.62it/s]


Discriminator Loss,█▄▄▅▅▄▃▄▃▄▂▂▂▃▂▂▂▂▃▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁
FID Score,▇█▆▇▆▆▅▆▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁
Generator Loss,▅▃▃▂▁▁▁▁▂▁▁▂▃▂▂▃▃▄▃▄▃▄▄▄▅▅▆▅▅▆▅▆▆▆▆▇▇▇██
Learning Rate Discriminator,█████████▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Learning Rate Generator,█████████▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Discriminator Loss,0.02002
FID Score,126.90077
Generator Loss,7.44185
Learning Rate Discriminator,3e-05
Learning Rate Generator,1e-05
